# Cats Vs Dogs

In [1]:
import numpy as np 
import pandas as pd 
import os
print(os.listdir("../Data"))

['Compressed', 'test', 'train']


In [2]:
import cv2                 
import numpy as np         
import os                  
from random import shuffle
from tqdm import tqdm      

train_dir = '../Data/train'
test_dir = '../Data/test'

In [3]:
def get_label(img):
    label = img.split('.')[0]
    if label == 'cat': 
        return [1,0]
    elif label == 'dog': 
        return [0,1]

**Y and test_y as one hot array.**

**Preprocessing and collecting train and test data.**

In [4]:
def making_train_data():
    training_data = []
    
    for img in tqdm(os.listdir(train_dir)):
        label = get_label(img)
        path = os.path.join(train_dir,img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (50,50))
        training_data.append([np.array(img),np.array(label)])
        
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data

In [5]:
def making_test_data():
    testing_data = []
    
    for img in tqdm(os.listdir(test_dir)):
        path = os.path.join(test_dir , img)
        img_num = img.split('.')[0]
        img = cv2.imread(path , cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img , (50,50))
        testing_data.append([np.array(img), img_num])
        
    shuffle(testing_data)
    np.save('test_data.npy', testing_data)
    return testing_data

In [6]:
train_data = making_train_data()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25000/25000 [01:36<00:00, 258.63it/s]


In [7]:
import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

Using TensorFlow backend.


In [8]:
train = train_data[0:20000]
test = train_data[20000:25000]

In [9]:
print(len(train) , len(test))

20000 5000


In [10]:
X = np.array([i[0] for i in train]).reshape(-1,1,50,50)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,1,50,50)
test_y = [i[1] for i in test]

**ImageDataGenerator to avoid overfitting the model with training data.**

In [11]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False,  
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False,  
        rotation_range=10,  
        zoom_range = 0.0,  
        width_shift_range=0.1,  
        height_shift_range=0.1,  
        horizontal_flip=False, 
        vertical_flip=False)  

datagen.fit(X)

**Using ReduceLROnPlateau to reduce the learning rate after certain epochs with callbacks.**

In [12]:
from keras.callbacks import ReduceLROnPlateau
lr_reduce = ReduceLROnPlateau(monitor='val_acc', factor=0.1, epsilon=0.0001, patience=1, verbose=1)

d:\python 3.7\lib\site-packages\keras\callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [13]:
Y = np.asarray(Y)
Y.reshape(len(Y) , 2)

array([[0, 1],
       [0, 1],
       [0, 1],
       ...,
       [1, 0],
       [1, 0],
       [0, 1]])

In [14]:
test_y = np.asarray(test_y)
test_y.reshape(len(test_y) , 2)

array([[0, 1],
       [0, 1],
       [1, 0],
       ...,
       [1, 0],
       [0, 1],
       [1, 0]])

In [15]:
test_x = test_x.reshape(-1, 1, 50, 50)

In [16]:
test_x = test_x / 255
X = X / 255

In [17]:
from keras.layers import Dense , Activation
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers import Conv2D , BatchNormalization
from keras.layers import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

In [18]:
def swish_activation(x):
    return (K.sigmoid(x) * x)

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', padding="same", input_shape=(1,50,50)))
model.add(Conv2D(32, (3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding="same"))
model.add(Conv2D(64, (3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(96, (3, 3), dilation_rate=(2, 2), activation='relu', padding="same"))
model.add(Conv2D(96, (3, 3), padding="valid", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), dilation_rate=(2, 2), activation='relu', padding="same"))
model.add(Conv2D(128, (3, 3), padding="valid", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64, activation=swish_activation))
model.add(Dropout(0.4))
model.add(Dense(2 , activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 50, 50)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 50, 50)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 25, 25)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 25, 25)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 25, 25)        36928     
____________________________________________________

In [19]:
batch_size = 128
epochs = 20

model.compile(loss='binary_crossentropy', optimizer='adam' , metrics=['accuracy'])
steps_per_epoch = len(train_data) // batch_size
validation_steps = len((test_x, test_y)) // batch_size

In [20]:
history = model.fit_generator(datagen.flow(X, Y, batch_size=batch_size),
                    steps_per_epoch=X.shape[0] // batch_size,
                    callbacks=[lr_reduce],
                    validation_data=(test_x, test_y),
                    epochs = epochs, verbose = 2)



Epoch 1/20
 - 39s - loss: 0.6883 - acc: 0.5305 - val_loss: 0.6770 - val_acc: 0.5727
Epoch 2/20
 - 19s - loss: 0.6665 - acc: 0.6002 - val_loss: 0.6469 - val_acc: 0.6215
Epoch 3/20
 - 18s - loss: 0.6209 - acc: 0.6592 - val_loss: 0.5619 - val_acc: 0.7131
Epoch 4/20
 - 18s - loss: 0.5667 - acc: 0.7089 - val_loss: 0.5298 - val_acc: 0.7356
Epoch 5/20
 - 18s - loss: 0.5060 - acc: 0.7546 - val_loss: 0.4520 - val_acc: 0.7830
Epoch 6/20
 - 19s - loss: 0.4609 - acc: 0.7814 - val_loss: 0.4184 - val_acc: 0.8069
Epoch 7/20
 - 19s - loss: 0.4252 - acc: 0.8032 - val_loss: 0.3887 - val_acc: 0.8245
Epoch 8/20
 - 19s - loss: 0.3851 - acc: 0.8236 - val_loss: 0.3808 - val_acc: 0.8282
Epoch 9/20
 - 18s - loss: 0.3679 - acc: 0.8337 - val_loss: 0.3622 - val_acc: 0.8367
Epoch 10/20
 - 19s - loss: 0.3504 - acc: 0.8412 - val_loss: 0.3471 - val_acc: 0.8415
Epoch 11/20
 - 18s - loss: 0.3317 - acc: 0.8543 - val_loss: 0.3450 - val_acc: 0.8505
Epoch 12/20
 - 19s - loss: 0.3116 - acc: 0.8626 - val_loss: 0.3586 - val

# Gaining a validation accuracy of 87.15 % with a self network architecture laid down.

# VGG-16 gave around 88.3 % as validation accuracy.

score = model.evaluate(test_x, test_y, verbose=0)
print('valid loss:', score[0])
print('valid accuracy:', score[1])

In [21]:
import matplotlib.pyplot as plt

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

**The Graphs are laid separately.**

In [22]:
test_data = making_test_data()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12500/12500 [00:54<00:00, 227.91it/s]


In [23]:
# with open('submission_file.csv','w') as f:
#     f.write('id,label\n')
            
# with open('submission_file.csv','a') as f:
#     for data in tqdm(test_data):
#         img_num = data[1]
#         img_data = data[0]
#         orig = img_data
#         data = img_data.reshape(1,1,50,50)
#         model_out = model.predict([data])[0]
#         f.write('{},{}\n'.format(img_num,model_out[1]))

**'submission_file' as the final submission file for the predictions in the test_data.**

In [24]:
model.save('../Model/Cats_and_Dogs.h5')

In [27]:
new_image_1 = cv2.imread('../cat.jpg', cv2.IMREAD_GRAYSCALE)
new_image_1 = cv2.resize(new_image_1 , (50,50))

cv2.imshow("Animal Picture", new_image_1)
cv2.waitKey(0)
cv2.destroyAllWindows()

resized_data = new_image_1.reshape(1,1,50,50)
iso_result = model.predict([resized_data])

# print(iso_result[0][0])
# print(iso_result[0][1])

result_animal = "Cat" if iso_result[0][0] > iso_result[0][1] else "Dog"
print("I think the animal is : ", result_animal)

I think the animal is :  Cat


In [28]:
# Short-Hand version of above cell...
new_image_2 = cv2.resize(cv2.imread('../cat.jpg', cv2.IMREAD_GRAYSCALE), (50, 50)).reshape(1, 1, 50, 50)

result_animal = "Cat" if iso_result[0][0] > iso_result[0][1] else "Dog"
print("I think the animal is : ", result_animal)

I think the animal is :  Cat


In [35]:
# trying to load model from disk...
from keras.models import load_model

model_2 = load_model("../Model/Cats_and_Dogs.h5", custom_objects = {"swish_activation": swish_activation})   
iso_result = model_2.predict([resized_data])
new_image_2 = cv2.resize(cv2.imread('../cat.jpg', cv2.IMREAD_GRAYSCALE), (50, 50)).reshape(1, 1, 50, 50) 
result_animal = "Cat" if iso_result[0][0] > iso_result[0][1] else "Dog"
print("I think the animal is : ", result_animal)


I think the animal is :  Cat
